In [2]:
import os
from pathlib import Path
# Setting the working directory to the root of the project
project_dir = Path("C:/Users/adbou/source/repos/KFHXRelatedAi/")
os.chdir(project_dir)

from Configs.GeneralPaths import SOURCEDATA
import pandas as pd

In [3]:
#loading the csv into pandas df

cleaned_deals_path = SOURCEDATA / "Cleaned_Deals.xlsx"

deals = pd.read_excel(cleaned_deals_path)

In [4]:
deals = deals.drop(columns=['Unnamed: 0'])
deals['Categories'] = deals['Categories'].replace('F&B', 'Food & Beverages')
deals.head()

,ContentId,Title,FK_StatusId,Deal Type,Description,Location,Points,Categories
0,113815,Off from Al Rifai,2,15 KD Voucher,Get a 15 KD discount voucher for your next onl...,https://www.google.com/maps/search/al+rifai+ku...,15000,Food & Beverages
1,113816,Off from Dip n dip,2,10 KD Voucher,Get 10 KD off on your next order from Dip n di...,NaN,10000,Food & Beverages
2,113817,Order from Dip n Dip and get discount,1,5KD discount,Order from Dip n Dip and get 5KD discount,https://www.google.com/maps/search/dipndip/@29...,5000,Food & Beverages
3,113818,Get discount from Ellena Bakery,2,3KD discount,Get 3KD discount from Ellena Bakery,https://maps.app.goo.gl/bnZXdfUbbaGFjFiJA,3000,Food & Beverages
4,113819,Off from Talabat,1,3 KD Voucher,Get 3KD discount from Talabat.,https://goo.gl/maps/5uoJEdD95W1YSpDt9,3000,Food & Beverages


In [5]:
from openai import OpenAI

client = OpenAI(api_key='sk-proj-lFCD6KXRS7R8kAaloineT3BlbkFJVlUaTzD3sIS4hzX9U0Fe')
EMBEDDING_MODEL = "text-embedding-3-small"

In [6]:
deals['TextEmbedding'] = deals['Title'] + ' ' + deals['Description'] + ' ' + deals['Categories']

deals = pd.DataFrame(deals)

In [7]:
#get embeddings

def get_Text_Embeddings(text):
    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text],model=EMBEDDING_MODEL).data[0].embedding


deals['ada_embedding'] = deals["TextEmbedding"].apply(lambda x: get_Text_Embeddings(x))

In [8]:
deals


,ContentId,Title,FK_StatusId,Deal Type,Description,Location,Points,Categories,TextEmbedding,ada_embedding
0,113815,Off from Al Rifai,2,15 KD Voucher,Get a 15 KD discount voucher for your next onl...,https://www.google.com/maps/search/al+rifai+ku...,15000,Food & Beverages,Off from Al Rifai Get a 15 KD discount voucher...,"[0.003274539252743125, 0.0044328998774290085, ..."
1,113816,Off from Dip n dip,2,10 KD Voucher,Get 10 KD off on your next order from Dip n di...,NaN,10000,Food & Beverages,Off from Dip n dip Get 10 KD off on your next ...,"[0.0014433693140745163, 0.004343641921877861, ..."
2,113817,Order from Dip n Dip and get discount,1,5KD discount,Order from Dip n Dip and get 5KD discount,https://www.google.com/maps/search/dipndip/@29...,5000,Food & Beverages,Order from Dip n Dip and get discount Order f...,"[0.0030049222987145185, -0.03488066792488098, ..."
3,113818,Get discount from Ellena Bakery,2,3KD discount,Get 3KD discount from Ellena Bakery,https://maps.app.goo.gl/bnZXdfUbbaGFjFiJA,3000,Food & Beverages,Get discount from Ellena Bakery Get 3KD disco...,"[-0.007099296431988478, -0.08420096337795258, ..."
4,113819,Off from Talabat,1,3 KD Voucher,Get 3KD discount from Talabat.,https://goo.gl/maps/5uoJEdD95W1YSpDt9,3000,Food & Beverages,Off from Talabat Get 3KD discount from Talabat...,"[-0.05206562951207161, -0.05629235878586769, 0..."
...,...,...,...,...,...,...,...,...,...,...
195,115227,Roblox $10 USA Gift Card,1,Roblox Gift Card,Get a $10 Roblox USA Gift Card to be used for ...,NaN,3500,Entertainment,Roblox $10 USA Gift Card Get a $10 Roblox USA ...,"[0.018920093774795532, 0.054280199110507965, -..."
196,115229,Amazon $10 USA Gift Card,1,Amazon Gift Card,Get a $10 Amazon&nbsp;USA Gift Card to be used...,NaN,3500,Entertainment,Amazon $10 USA Gift Card Get a $10 Amazon&nbsp...,"[0.028729772195219994, 0.01610574498772621, -0..."
197,115231,PlayStation Kuwait $10 Gift Card,1,PlayStation Kuwait Gift Card,Get a $10 PlayStation Kuwait&nbsp;Gift Card to...,NaN,3500,Entertainment,PlayStation Kuwait $10 Gift Card Get a $10 Pla...,"[0.0058990903198719025, -0.014113013632595539,..."
198,115244,Recharge your Deliveroo wallet with,2,10KD Wallet Recharge,Recharge your Deliveroo wallet with 10KD,https://www.instagram.com/deliveroo_kw/,1000,Food & Beverages,Recharge your Deliveroo wallet with Recharge y...,"[-0.0007109831785783172, -0.02498771622776985,..."
